<a href="https://colab.research.google.com/github/mistryvivek/flowers-nn-int2/blob/main/Model2SimpleWithPaddingChanged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
# Download training data from open datasets.
training_data = datasets.Flowers102(
    root="data",
    split="train",
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()]),
)

# Download test data from open datasets.
test_data = datasets.Flowers102(
    root="data",
    split="test",
    download=True,
    transform = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()]),
)

100%|██████████| 344862509/344862509 [00:16<00:00, 21383376.36it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 481266.42it/s]


100%|██████████| 14989/14989 [00:00<00:00, 14272059.63it/s]


In [3]:
batch_size = 30
num_classes = 102
learning_rate = 0.001
num_epochs = 50

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [4]:
import torch.nn.functional as F

class Net(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding_mode='replicate')
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding_mode='replicate')
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding_mode='replicate')
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding_mode='replicate')
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out


net = Net(num_classes)

In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i, (images, labels) in enumerate(train_dataloader):  
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = net(images)
        loss = criterion(outputs, labels)
      
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/50], Loss: 4.6331
Epoch [2/50], Loss: 4.6333
Epoch [3/50], Loss: 4.6337
Epoch [4/50], Loss: 4.6340
Epoch [5/50], Loss: 4.6337
Epoch [6/50], Loss: 4.6337
Epoch [7/50], Loss: 4.6339
Epoch [8/50], Loss: 4.6338
Epoch [9/50], Loss: 4.6336
Epoch [10/50], Loss: 4.6333
Epoch [11/50], Loss: 4.6328
Epoch [12/50], Loss: 4.6323
Epoch [13/50], Loss: 4.6315
Epoch [14/50], Loss: 4.6305
Epoch [15/50], Loss: 4.6293
Epoch [16/50], Loss: 4.6277
Epoch [17/50], Loss: 4.6262
Epoch [18/50], Loss: 4.6243
Epoch [19/50], Loss: 4.6219
Epoch [20/50], Loss: 4.6190
Epoch [21/50], Loss: 4.6153
Epoch [22/50], Loss: 4.6109
Epoch [23/50], Loss: 4.6045
Epoch [24/50], Loss: 4.5967
Epoch [25/50], Loss: 4.5840
Epoch [26/50], Loss: 4.5666
Epoch [27/50], Loss: 4.5381
Epoch [28/50], Loss: 4.4885
Epoch [29/50], Loss: 4.4320
Epoch [30/50], Loss: 4.3847
Epoch [31/50], Loss: 4.3537
Epoch [32/50], Loss: 4.3063
Epoch [33/50], Loss: 4.2512
Epoch [34/50], Loss: 4.2006
Epoch [35/50], Loss: 4.1315
Epoch [36/50], Loss: 3.5351
E

In [7]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Accuracy of the network on the {} train images: {} %'.format(total, 100 * correct / total))

Accuracy of the network on the 6149 train images: 9.383639616197756 %
